In [1]:
from flask import Flask, jsonify, render_template
import numpy as np
import datetime as dt
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from config import password
import pandas as pd
import joblib


In [2]:
# connection_string = f"postgres:{password}@perth-property-market.cptzycsh4y3w.ap-southeast-2.rds.amazonaws.com:5432/perth_property_market"

# # Create the database engine (to the PostgreSQL database)
# engine = create_engine(f'postgresql://{connection_string}')
# conn = engine.connect()
# session = Session(bind=engine)

In [3]:
# Base = automap_base()

# Base.prepare(engine, reflect=True)

# market_data = Base.classes.perth_market

In [4]:
# properties = session.query(
#     market_data.price, market_data.bedrooms, market_data.bathrooms,
#     market_data.land_area, market_data.build_year, market_data.cbd_dist).all()

In [5]:
# session.close()


In [6]:
# property_list = []
# for row in properties:
#     property = list(np.ravel(row))
#     property_list.append(property)
    

# df = pd.DataFrame(property_list, columns=["Price", "Bedrooms", "Bathrooms", "Land_Area", "Build_Year", 
#     "CBD_Dist"])
# df

In [7]:
df = pd.read_csv("../csv/cleaned_price_categorical.csv")
df

,ADDRESS,SUBURB,PRICE,BEDROOMS,BATHROOMS,GARAGE,LAND_AREA,FLOOR_AREA,BUILD_YEAR,CBD_DIST,NEAREST_STN,NEAREST_STN_DIST,POSTCODE,LATITUDE,LONGITUDE,NEAREST_SCH,NEAREST_SCH_DIST,YEAR_SOLD,MONTH_SOLD
0,"1 Acorn Place, South Lake",South Lake,550-600k,4,2,2,600,160,2003,18300,Cockburn Central Station,1800,6164,-32.115900,115.842450,LAKELAND SENIOR HIGH SCHOOL,0.828339,2018,9
1,"1 Addis Way, Wandi",Wandi,350-400k,3,2,2,351,139,2013,26900,Kwinana Station,4900,6167,-32.193470,115.859554,ATWELL COLLEGE,5.524324,2019,2
2,"1 Ainsley Court, Camillo",Camillo,250-300k,3,1,1,719,86,1979,22600,Challis Station,1900,6111,-32.120578,115.993579,KELMSCOTT SENIOR HIGH SCHOOL,1.649178,2015,6
3,"1 Albert Street, Bellevue",Bellevue,250-300k,2,1,2,651,59,1953,17900,Midland Station,3600,6056,-31.900547,116.038009,SWAN VIEW SENIOR HIGH SCHOOL,1.571401,2018,7
4,"1 Aman Place, Lockridge",Lockridge,300-350k,4,1,2,466,131,1998,11200,Bassendean Station,2000,6054,-31.885790,115.947780,KIARA COLLEGE,1.514922,2016,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25678,"9B Teocote Crescent, Aubin Grove",Aubin Grove,400-450k,3,2,2,248,108,2011,24900,Cockburn Central Station,5700,6164,-32.176013,115.858357,ATWELL COLLEGE,3.607062,2016,5
25679,"9B Viveash Road, Middle Swan",Middle Swan,450-500k,4,2,2,400,132,2014,17100,Midland Station,2500,6056,-31.872192,116.013543,LA SALLE COLLEGE,1.116806,2015,4
25680,"9b Wesley Street, South Fremantle",South Fremantle,950-1000k,3,2,1,200,127,1997,16000,Fremantle Station,1900,6162,-32.066410,115.756070,FREMANTLE COLLEGE,1.134956,2019,2
25681,"9C Gold Street, South Fremantle",South Fremantle,1000-1050k,4,3,2,292,245,2013,16100,Fremantle Station,1500,6162,-32.064580,115.751820,CHRISTIAN BROTHERS' COLLEGE,1.430350,2016,3


In [8]:
df_for_dummies = df[["BEDROOMS", "BATHROOMS", "LAND_AREA", "FLOOR_AREA", "BUILD_YEAR", 
"CBD_DIST", "YEAR_SOLD", "PRICE"]]

df_for_dummies.head(2)

,BEDROOMS,BATHROOMS,LAND_AREA,FLOOR_AREA,BUILD_YEAR,CBD_DIST,YEAR_SOLD,PRICE
0,4,2,600,160,2003,18300,2018,550-600k
1,3,2,351,139,2013,26900,2019,350-400k


# Decision Tree

In [9]:
from sklearn import tree


In [10]:
X = df[["BEDROOMS", "BATHROOMS", "LAND_AREA", "FLOOR_AREA", "BUILD_YEAR", 
"CBD_DIST", "YEAR_SOLD"]]

y = df["PRICE"]

print(X.shape, y.shape)

(25683, 7) (25683,)


In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [12]:
data = df_for_dummies.values
X = data[:,0:7]
y = data[:,7]

In [20]:
X

array([[4, 2, 600, ..., 2003, 18300, 2018],
       [3, 2, 351, ..., 2013, 26900, 2019],
       [3, 1, 719, ..., 1979, 22600, 2015],
       ...,
       [3, 2, 200, ..., 1997, 16000, 2019],
       [4, 3, 292, ..., 2013, 16100, 2016],
       [3, 1, 457, ..., 1974, 12600, 2016]], dtype=object)

In [13]:
from sklearn.preprocessing import LabelEncoder

# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y)
encoded_y = label_encoder.transform(y)

In [14]:
for label, original_class in zip(encoded_y, y):
    print('Original Class: ' + str(original_class))
    print('Encoded Label: ' + str(label))
    print('-' * 12)

Original Class: 550-600k
Encoded Label: 38
------------
Original Class: 350-400k
Encoded Label: 34
------------
Original Class: 250-300k
Encoded Label: 32
------------
Original Class: 250-300k
Encoded Label: 32
------------
Original Class: 300-350k
Encoded Label: 33
------------
Original Class: 400-450k
Encoded Label: 35
------------
Original Class: 350-400k
Encoded Label: 34
------------
Original Class: 350-400k
Encoded Label: 34
------------
Original Class: 550-600k
Encoded Label: 38
------------
Original Class: 650-700k
Encoded Label: 40
------------
Original Class: 350-400k
Encoded Label: 34
------------
Original Class: 250-300k
Encoded Label: 32
------------
Original Class: 500-550k
Encoded Label: 37
------------
Original Class: 500-550k
Encoded Label: 37
------------
Original Class: 650-700k
Encoded Label: 40
------------
Original Class: 300-350k
Encoded Label: 33
------------
Original Class: 550-600k
Encoded Label: 38
------------
Original Class: 300-350k
Encoded Label: 33
-----

In [15]:
from keras.utils import to_categorical

# Step 2: One-hot encoding
one_hot_y = to_categorical(encoded_y)
one_hot_y

Using TensorFlow backend.


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,one_hot_y)

In [17]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)


0.1778539168353839

In [21]:
clf.score(X_train, y_train)

0.9997923372443153

In [18]:
feature_names = df_for_dummies.columns
sorted(zip(clf.feature_importances_, feature_names), reverse=True)

[(0.2294882172204252, 'CBD_DIST'),
 (0.21572683324388972, 'LAND_AREA'),
 (0.20509244039853458, 'FLOOR_AREA'),
 (0.16802401152777252, 'BUILD_YEAR'),
 (0.1131256279448357, 'YEAR_SOLD'),
 (0.04359881253697688, 'BEDROOMS'),
 (0.024944057127565353, 'BATHROOMS')]

# Random Forest

In [19]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.035819965737424075

In [22]:
rf.score(X_train, y_train)

0.9998442529332364

# Neural Networks with Keras

In [18]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)

In [19]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [20]:
from tensorflow.keras.utils import to_categorical

In [21]:
from tensorflow.keras.models import Sequential

model = Sequential()

In [22]:
len(X_train_scaled[0])

7

In [23]:
from tensorflow.keras.layers import Dense
number_inputs = len(X_train_scaled[0])
number_hidden_nodes = 2
model.add(Dense(units=number_hidden_nodes,
                activation='relu', input_dim=number_inputs))

In [24]:
number_classes = len(one_hot_y)
model.add(Dense(units=number_classes, activation='softmax'))

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 2)                 16        
_________________________________________________________________
dense_1 (Dense)              (None, 25683)             77049     
Total params: 77,065
Trainable params: 77,065
Non-trainable params: 0
_________________________________________________________________


## Compile the Model

In [26]:
# Use categorical crossentropy for categorical data and mean squared error for regression
# Hint: your output layer in this example is using software for logistic regression (categorical)
# If your output layer activation was `linear` then you may want to use `mse` for loss
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [27]:
# Fit (train) the model
model.fit(
    X_train_scaled,
    one_hot_y,
    epochs=10,
    shuffle=True,
    verbose=2
)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Nassim\anaconda3\envs\PythonAdv\lib\site-packages\IPython\core\interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-27-4aa6fc9be29d>", line 7, in <module>
    verbose=2
  File "C:\Users\Nassim\anaconda3\envs\PythonAdv\lib\site-packages\tensorflow_core\python\keras\engine\training.py", line 819, in fit
    use_multiprocessing=use_multiprocessing)
  File "C:\Users\Nassim\anaconda3\envs\PythonAdv\lib\site-packages\tensorflow_core\python\keras\engine\training_v2.py", line 235, in fit
    use_multiprocessing=use_multiprocessing)
  File "C:\Users\Nassim\anaconda3\envs\PythonAdv\lib\site-packages\tensorflow_core\python\keras\engine\training_v2.py", line 593, in _process_training_inputs
    use_multiprocessing=use_multiprocessing)
  File "C:\Users\Nassim\anaconda3\envs\PythonAdv\lib\site-packages\tensorflow_core\python\keras\engine\training_v2.py", line 646, in _process_inpu

## Saving Model

In [74]:
# save the model to disk
filename = 'property_model_decisionTree.sav'
joblib.dump(clf, filename)


['property_model_decisionTree.sav']

In [76]:
# load the model from disk
loaded_model = joblib.load(filename)
result = loaded_model.score(X_test_scaled, y_test_scaled)
print(result)

-0.043811568858406424
